In [6]:
%pip install flask-restful

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from flask import Flask
from flask_restful import Api
app = Flask(__name__)
api = Api(app)

In [8]:
from flask import jsonify
from flask_restful import Resource

BOOKS = {
    "1": {"title": "The Hitchhiker's Guide to the Galaxy", "author": "Douglas Adams", "publication_year": 1979},
    "2": {"title": "1984", "author": "George Orwell", "publication_year": 1949},
    "3": {"title": "To Kill a Mockingbird", "author": "Harper Lee", "publication_year": 1960}
}

class Books(Resource):
    def get(self):
        return jsonify(BOOKS)

In [9]:
from flask import request

class Books(Resource):
    def get(self):
        search_query = request.args.get('search')
        if search_query:
            results = [book for book in BOOKS.values() if
                       search_query.lower() in book['title'].lower() or
                       search_query.lower() in book['author'].lower()]
            return jsonify(results)
        return jsonify(BOOKS)

In [10]:
from flask_restful.reqparse import RequestParser
from flask import abort

parser = RequestParser()
parser.add_argument('title', type=str, required=True, help="Title is required")
parser.add_argument('author', type=str, required=True, help="Author is required")
parser.add_argument('publication_year', type=int, required=True, help="Publication year is required")

def authenticate(username, password):
    return username == 'admin' and password == 'password'

class Books(Resource):
    def post(self):
        auth = request.authorization
        if not auth or not authenticate(auth.username, auth.password):
            abort(401, description="Authentication required")

        args = parser.parse_args()
        new_id = str(int(max(BOOKS.keys(), default=0)) + 1)
        BOOKS[new_id] = {
            'title': args['title'],
            'author': args['author'],
            'publication_year': args['publication_year']
        }
        return jsonify(BOOKS[new_id])

In [11]:
class Book(Resource):
    def get(self, book_id):
        if book_id not in BOOKS:
            abort(404, description="Book not found")
        return jsonify(BOOKS[book_id])

In [12]:
api.add_resource(Books, '/books/')
api.add_resource(Book, '/books/<string:book_id>')

In [ ]:
from threading import Thread

def run_app():
    app.run(debug=True, use_reloader=False)

thread = Thread(target=run_app)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Feb/2026 19:49:04] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [07/Feb/2026 19:49:05] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Feb/2026 19:49:21] "GET /books/ HTTP/1.1" 405 -
127.0.0.1 - - [07/Feb/2026 19:50:13] "GET /books/1 HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2026 19:50:15] "GET /books/2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2026 19:50:18] "GET /books/3 HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2026 19:50:38] "GET /books HTTP/1.1" 404 -
127.0.0.1 - - [07/Feb/2026 19:50:41] "GET /books/1 HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2026 19:50:43] "GET /books/ HTTP/1.1" 405 -
127.0.0.1 - - [07/Feb/2026 19:51:38] "GET /books/10 HTTP/1.1" 404 -
